# Pandas groupby on parquet with numpy vs arrow

In [1]:
import pandas as pd
import polars as pl
from humanfriendly import format_size, format_number
from simpler_mpl import set_commas, set_common_mpl_styles
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

display(f"Pandas {pd.__version__}, Polars {pl.__version__}")

from cell_profiler import cell_profiler as cp
%start_cell_profiler

def show_rows_memory(df, deep=False):
    """
    Show rows and memory cost of a Pandas/Polars dataframe
    `deep=True` only has an impact on Pandas numpy-backed string columns, which otherwise are undercounted
    """
    num_bytes = 0
    df_type = 'Unknown'
    try:
        num_bytes = df.estimated_size() # try Polars
        df_type = 'Polars'
    except AttributeError:
        pass
    try:
        num_bytes = df.memory_usage(deep=deep, index=False).sum() # try Pandas
        df_type = 'Pandas'
    except AttributeError:
        pass
    display(f"{df_type} df with {format_number(df.shape[0])} rows, {format_size(num_bytes)} bytes")

'Pandas 2.0.1, Polars 0.17.10'

In [1] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 207.7 MiB


In [2]:
dfpda = pd.read_parquet('../test_result_2021on.parquet', dtype_backend="pyarrow")

In [2] used 12897.0 MiB RAM in 8.97s (system mean cpu 41%, single max cpu 100%), peaked 3500.9 MiB above final usage, current RAM usage now 13104.7 MiB


In [15]:
dfpda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82012245 entries, 0 to 82012244
Data columns (total 14 columns):
 #   Column             Dtype                 
---  ------             -----                 
 0   test_id            int64[pyarrow]        
 1   vehicle_id         int64[pyarrow]        
 2   test_date          timestamp[ns][pyarrow]
 3   test_class_id      int64[pyarrow]        
 4   test_type          large_string[pyarrow] 
 5   test_result        large_string[pyarrow] 
 6   test_mileage       int64[pyarrow]        
 7   postcode_area      large_string[pyarrow] 
 8   make               large_string[pyarrow] 
 9   model              large_string[pyarrow] 
 10  colour             large_string[pyarrow] 
 11  fuel_type          large_string[pyarrow] 
 12  cylinder_capacity  int64[pyarrow]        
 13  first_use_date     timestamp[ns][pyarrow]
dtypes: int64[pyarrow](5), large_string[pyarrow](7), timestamp[ns][pyarrow](2)
memory usage: 10.4 GB
In [15] used 4.7 MiB RAM in 1.33

In [3]:
mask = dfpda['cylinder_capacity'].isna()
mask.sum() # nbr of rows to remove

242604

In [3] used 25.0 MiB RAM in 0.82s (system mean cpu 8%, single max cpu 100%), peaked 0.7 MiB above final usage, current RAM usage now 13129.7 MiB


In [4]:
df_nonan_cylcap = dfpda[~mask]
df_nonan_cylcap.shape

(81769641, 14)

In [4] used 11957.8 MiB RAM in 19.32s (system mean cpu 8%, single max cpu 100%), peaked 1311.0 MiB above final usage, current RAM usage now 25087.5 MiB


In [11]:
#agg_result = df_nonan_cylcap.groupby('make')['cylinder_capacity'].agg(['median', 'size'])
#agg_result.query('size > 10').sort_values('median')
#agg_result.query('size == 1').sort_index()

In [11] used -0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 36922.8 MiB


In [20]:
#mask = dfpda.make.str.startswith('TESLA')
#dfpda[mask] # so TESLA 40k rows, but NA cylinder_capacity and fuel_type Electric or EL!

In [20] used -0.1 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 36919.4 MiB


In [28]:
%%time

#result = dfpda[~dfpda['cylinder_capacity'].isna()] \ # same speed

result = dfpda.dropna(subset=['cylinder_capacity']) \
.groupby('make')['cylinder_capacity'].agg(['median', 'count']) \
.query('count > 10').sort_values('median')
result

CPU times: user 12.4 s, sys: 6.33 s, total: 18.7 s
Wall time: 18.6 s


,median,count
make,,
TESLA,0.0,16
VECTRIX,4.0,13
CPI,49.0,563
MOTOBI,49.0,54
MOTOR HISPANIA,49.0,92
...,...,...
ERF,10000.0,30
LEYLAND NATIONAL,10450.0,15
NEOPLAN,12609.5,18


In [28] used 2.9 MiB RAM in 18.71s (system mean cpu 10%, single max cpu 100%), peaked 15199.8 MiB above final usage, current RAM usage now 19595.9 MiB


In [29]:
assert result.loc['ROLLS ROYCE']['median'] == 6749.0
assert result.loc['ROLLS ROYCE']['count'] == 11741.0

In [29] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 19595.9 MiB


In [31]:
#mask = result.index.str.startswith('ROLL')
#result[mask]

In [31] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 19595.9 MiB


# Repeat for numpy equivalent

In [9]:
dfpdn = pd.read_parquet('../test_result_2021on.parquet', dtype_backend="numpy_nullable")

In [9] used 22793.9 MiB RAM in 27.27s (system mean cpu 22%, single max cpu 100%), peaked 665.8 MiB above final usage, current RAM usage now 47895.8 MiB


In [34]:
%%time
# result = dfpdn[~dfpdn['cylinder_capacity'].isna()] \
result = dfpdn.dropna(subset=['cylinder_capacity']) \
.groupby('make')['cylinder_capacity'].agg(['median', 'count']) \
.query('count > 10').sort_values('median')
result

CPU times: user 18.6 s, sys: 4.63 s, total: 23.2 s
Wall time: 23.1 s


,median,count
make,,
TESLA,0.0,16
VECTRIX,4.0,13
CPI,49.0,563
MOTOBI,49.0,54
MOTOR HISPANIA,49.0,92
...,...,...
ERF,10000.0,30
LEYLAND NATIONAL,10450.0,15
NEOPLAN,12609.5,18


In [34] used -25.7 MiB RAM in 23.19s (system mean cpu 9%, single max cpu 100%), peaked 11853.0 MiB above final usage, current RAM usage now 23841.5 MiB


In [35]:
assert result.loc['ROLLS ROYCE']['median'] == 6749.0
assert result.loc['ROLLS ROYCE']['count'] == 11741.0

In [35] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 23841.5 MiB
